In [2]:
import numpy as np

In [3]:
import nltk

In [4]:
from nltk import word_tokenize

In [5]:
import re

In [6]:
import pandas as pd

In [7]:
f = open("amazon_cells_labelled.txt")

In [8]:
f2 = open("imdb_labelled.txt")

In [9]:
f3 = open("yelp_labelled_fix_test.txt")

In [10]:
f4 = open("train_data.txt")

In [11]:
str_list = ["" for i in range(1000)]
for line,i in zip(f,range(1000)):
    str_list[i] = re.sub('[.,]','',line)

In [12]:
str_list2 = ["" for i in range(1000)]
for line,i in zip(f2,range(1000)):
    str_list2[i] = re.sub('[.,]','',line)

In [13]:
str_list3 = ["" for i in range(11000)]
for line,i in zip(f3,range(11000)):
    str_list3[i] = re.sub('[.,]','',line)

In [14]:
length = 575
str_list4 = ["" for i in range(length)]
for line,i in zip(f4,range(length)):
    str_list4[i] = re.sub('[.,]','',line)

In [15]:
str_super = str_list + str_list2 + str_list3 + str_list4

In [16]:
test_list = str_list[500:]

In [17]:
for i in range(len(str_super)):
    str_super[i] = str_super[i].strip('\t0\n')
    str_super[i] = str_super[i].strip('\t1\n')
    str_super[i] = str_super[i].lower()

In [18]:
for i in range(len(test_list)):
    test_list[i] = test_list[i].strip('\t0\n')
    test_list[i] = test_list[i].strip('\t1\n')
    test_list[i] = test_list[i].lower()

In [19]:
token_list = [[] for i in range(1000)]
for i in range(1000):
    token_list[i] = word_tokenize(str_list[i])

In [20]:
token_list2 = [[] for i in range(1000)]
for i in range(1000):
    token_list2[i] = word_tokenize(str_list2[i])

In [21]:
token_list3 = [[] for i in range(11000)]
for i in range(11000):
    token_list3[i] = word_tokenize(str_list3[i])

In [22]:
token_list4 = [[] for i in range(length)]
for i in range(length):
    token_list4[i] = word_tokenize(str_list4[i])

In [23]:
label = [token_list[i][len(token_list[i]) - 1] for i in range(1000)]

In [24]:
label2 = [token_list2[i][len(token_list2[i]) - 1] for i in range(1000)]

In [25]:
label3 = [token_list3[i][len(token_list3[i]) - 1] for i in range(11000)]

In [26]:
label4 = [token_list4[i][- 1] for i in range(length)]

In [27]:
super_label = label[:500] + label2 + label3 + label4

In [28]:
test_label = label[500:]

In [29]:
super_label = ['1' if super_label[i] == '1' else '-1' for i in range(len(super_label))]

In [30]:
test_label = ['1' if test_label[i] == '1' else '-1' for i in range(len(test_label))]

In [31]:
str_data = [token_list[i][:len(token_list[i]) - 1] for i in range(1000)]

In [32]:
str_data = [[str_data[i][j].lower() for j in range(len(str_data[i]))] for i in range(1000)]

In [33]:
str_data2 = [token_list2[i][:len(token_list2[i]) - 1] for i in range(1000)]

In [34]:
str_data2 = [[str_data2[i][j].lower() for j in range(len(str_data2[i]))] for i in range(1000)]

In [35]:
str_data3 = [token_list3[i][:len(token_list3[i]) - 1] for i in range(11000)]

In [36]:
str_data3 = [[str_data3[i][j].lower() for j in range(len(str_data3[i]))] for i in range(11000)]

In [37]:
str_data4 = [token_list4[i][:len(token_list4[i]) - 1] for i in range(length)]

In [38]:
str_data4 = [[str_data4[i][j].lower() for j in range(len(str_data4[i]))] for i in range(length)]

In [39]:
str_super_data = str_data[:500] + str_data2 + str_data3 + str_data4

In [40]:
test_str_data = str_data[500:]

In [41]:
from collections import Counter

In [42]:
super_positive = [str_super_data[i] if super_label[i] == '1' else None for i in range(len(str_super_data))]

In [43]:
super_negative = [str_super_data[i] if super_label[i] == '-1' else None for i in range(len(str_super_data))]

In [44]:
super_pos = Counter([])
for i in range(len(super_positive)):
    super_pos += Counter(super_positive[i])

In [45]:
super_neg = Counter([])
for i in range(len(super_negative)):
    super_neg += Counter(super_negative[i])

In [46]:
super_pos = dict(super_pos)
super_neg = dict(super_neg)

In [47]:
pos_vocab = list(super_pos.keys())
neg_vocab = list(super_neg.keys())

In [48]:
num_vocab = 35000

In [49]:
def baseline(inptstr, phi_pos, phi_neg, phi_ypos, phi_yneg, num_vocab, pos_voc, neg_voc):
    tok_list = word_tokenize(inptstr)
    log_prob_pos = 0
    log_prob_neg = 0
    for word in tok_list:
        if(word in pos_voc):
            log_prob_pos += np.log(phi_pos[word])
        else:
            log_prob_pos += np.log(1/num_vocab)
    for word in tok_list:
        if(word in neg_voc):
            log_prob_neg += np.log(phi_neg[word])
        else:
            log_prob_neg += np.log(1/num_vocab)
    log_prob_pos += np.log(phi_ypos)
    log_prob_neg += np.log(phi_yneg)
    if(log_prob_pos >= log_prob_neg):
        return '1'
    else:
        return '-1'

In [50]:
def weight_train(s_data, l_data, pos_voc, neg_voc, weight):
    data_size = len(s_data)
    tot_weight = sum([weight[i] for i in range(data_size)])
    weight_pos = sum([weight[i] if l_data[i] == '1' else 0 for i in range(data_size)])
    weight_neg = sum([weight[i] if l_data[i] == '-1' else 0 for i in range(data_size)])
    phi_ypos = weight_pos/tot_weight
    phi_yneg = weight_neg/tot_weight
    phi_pos_num = {i:0 for i in pos_voc}
    phi_neg_num = {i:0 for i in neg_voc}
    for i in range(data_size):
        str_size = len(s_data[i])
        if(l_data[i] == '1'):
            for j in range(str_size):
                phi_pos_num[s_data[i][j]] += weight[i]
        else:
            for j in range(str_size):
                phi_neg_num[s_data[i][j]] += weight[i]
    phi_pos = {i:phi_pos_num[i]/weight_pos for i in pos_voc}
    phi_neg = {i:phi_neg_num[i]/weight_neg for i in neg_voc}
    return phi_pos, phi_neg, phi_ypos, phi_yneg

In [51]:
def error(s_raw, s_data, l_data, phi_pos, phi_neg, phi_ypos, phi_yneg, weight, num_vocab, pos_voc, neg_voc):
    prediction = predict(s_raw, s_data, phi_pos, phi_neg, phi_ypos, phi_yneg, num_vocab, pos_voc, neg_voc)
    data_size = len(s_data)
    weighted_error = sum([weight[i] if prediction[i] != l_data[i] else 0 for i in range(data_size)])
    return weighted_error,prediction

In [52]:
def predict(s_raw, s_data, phi_pos, phi_neg, phi_ypos, phi_yneg, num_vocab, pos_voc, neg_voc):
    data_size = len(s_data)
    prediction = [baseline(s_raw[i], phi_pos, phi_neg, phi_ypos, phi_yneg, num_vocab, pos_voc, neg_voc) for i in range(data_size)]
    return prediction

In [53]:
def boosting(s_raw, s_data, l_data, pos_voc, neg_voc, steps, num_vocab):
    weight = [1/len(s_data) for i in range(len(s_data))]
    beta = [0 for i in range(steps)]
    phi_pos = [{} for i in range(steps)]
    phi_neg = [{} for i in range(steps)]
    phi_ypos = [0 for i in range(steps)]
    phi_yneg = [0 for i in range(steps)]
    for i in range(steps):
        print('step',i)
        phi_pos[i], phi_neg[i], phi_ypos[i], phi_yneg[i] = weight_train(s_data, l_data, pos_voc, neg_voc, weight)
        weighted_error,prediction = error(s_raw, s_data,l_data,phi_pos[i],phi_neg[i],phi_ypos[i],phi_yneg[i], weight, num_vocab,pos_voc, neg_voc)
        beta[i] = 0.5*np.log((1 - weighted_error)/weighted_error)
        for j in range(len(s_data)):
            weight[j] = weight[j]*np.exp(-beta[i]*float(l_data[j])*float(prediction[j]))
    return beta, phi_pos, phi_neg, phi_ypos, phi_yneg

In [54]:
def boost_predict(inptstr, beta, phi_pos, phi_neg, phi_ypos, phi_yneg, num_vocab, pos_voc, neg_voc):
    pred = 0
    for i in range(len(beta)):
        pred += beta[i]*float(baseline(inptstr, phi_pos[i], phi_neg[i], phi_ypos[i], phi_yneg[i], num_vocab, pos_voc, neg_voc))
    if(pred >= 0):
        return '1'
    else:
        return '-1'

In [55]:
beta, phi_pos, phi_neg, phi_ypos, phi_yneg = boosting(str_super, str_super_data, super_label, pos_vocab, neg_vocab,10, num_vocab)

step 0
step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9


In [58]:
test_len = 500
train_len = len(super_label)
count = [0 for i in range(10)]
ncount = [0 for i in range(10)]
neg_count = [0 for i in range(10)]
for k in range(10):
    for i in range(train_len):
        if super_label[i] == boost_predict(str_super[i],beta[:k + 1], phi_pos[:k + 1], phi_neg[:k + 1], phi_ypos[:k + 1], phi_yneg[:k + 1], num_vocab, pos_vocab, neg_vocab):
            count[k] += 1
            if super_label[i] == '-1':
                neg_count[k] += 1
        if k >= 1:
            continue
        if super_label[i] == baseline(str_super[i],phi_pos[0],phi_neg[0],phi_ypos[0],phi_yneg[0], num_vocab,pos_vocab, neg_vocab):
            ncount[k] += 1
neg_len = super_label.count('-1')
for k in range(10):
    print(neg_count[k]/neg_len)
    print(count[k]/train_len)
    print(ncount[k]/train_len)

0.9241301546391752
0.9189292543021033
0.9189292543021033
0.9241301546391752
0.9189292543021033
0.0
0.8155605670103093
0.9069216061185469
0.0
0.998389175257732
0.9196940726577438
0.0
0.9747100515463918
0.935678776290631
0.0
0.8170103092783505
0.9083747609942638
0.0
0.8356958762886598
0.9190057361376673
0.0
0.9975837628865979
0.9320076481835564
0.0
0.9758376288659794
0.9380497131931166
0.0
0.8584085051546392
0.92848948374761
0.0


In [60]:
count = [0 for i in range(10)]
ncount = [0 for i in range(10)]
neg_count = [0 for i in range(10)]
for k in range(10):
    for i in range(test_len):
        if test_label[i] == boost_predict(test_list[i],beta[:k + 1], phi_pos[:k + 1], phi_neg[:k + 1], phi_ypos[:k + 1], phi_yneg[:k + 1], num_vocab, pos_vocab, neg_vocab):
            count[k] += 1
            if test_label[i] == '-1':
                neg_count[k] += 1
        if k >= 1:
            continue
        if test_label[i] == baseline(test_list[i],phi_pos[0],phi_neg[0],phi_ypos[0],phi_yneg[0], num_vocab,pos_vocab, neg_vocab):
            ncount[k] += 1
neg_len = test_label.count('-1')
for k in range(10):
    print(neg_count[k]/neg_len)
    print(count[k]/test_len)
    print(ncount[k]/test_len)

0.8045977011494253
0.738
0.738
0.8045977011494253
0.738
0.0
0.5210727969348659
0.69
0.0
0.842911877394636
0.7
0.0
0.8237547892720306
0.732
0.0
0.5210727969348659
0.69
0.0
0.5172413793103449
0.694
0.0
0.8275862068965517
0.716
0.0
0.8237547892720306
0.734
0.0
0.5287356321839081
0.692
0.0


In [62]:
super_label.count('-1')

6208

In [185]:
boost_predict("", beta, phi_pos, phi_neg, phi_ypos, phi_yneg, num_vocab, pos_vocab, neg_vocab)

'-1'

In [1]:
baseline("fuck", phi_pos[0], phi_neg[0], phi_ypos[0], phi_yneg[0], num_vocab, pos_vocab, neg_vocab)

NameError: name 'baseline' is not defined

In [186]:
for i in range(9):
    print(baseline("", phi_pos[i], phi_neg[i], phi_ypos[i], phi_yneg[i], num_vocab, pos_vocab, neg_vocab))
    print(beta[i])

-1
1.20781247091
-1
1.11577524526
-1
1.09027257771
-1
1.33785112918
-1
0.973972541647
1
1.37524715268
1
0.955980953122
-1
1.50900258262
-1
1.0273989497


In [64]:
def threshold(value): ##return true by probability of value
    rnd = np.random.rand()
    if rnd < value:
        return True
    else:
        return False

In [65]:
def gibbs_sample(phi_pos, phi_neg, phi_ypos, phi_yneg, pos_voc, neg_voc, num_sample):
    for smp in range(num_sample):
        if(threshold(phi_ypos)):
            print("positive:")
            for i in range(len(pos_voc)):
                if(threshold(phi_pos[pos_voc[i]])):
                    print(pos_voc[i],end = ' ')
            print()
        else:
            print("negative:")
            for i in range(len(neg_voc)):
                if(threshold(phi_neg[neg_voc[i]])):
                    print(neg_voc[i],end = ' ')
            print()

In [73]:
gibbs_sample(phi_pos[9],phi_neg[9],phi_ypos[9],phi_yneg[9],pos_vocab, neg_vocab, 100)

negative:
the i my and that their sitting movie video cute indie seldom 
positive:
for if this with all feels more your anything among situations scary karmen juice lump 
negative:
it be that just nothing core making affair 
positive:
you very to one it say of on they editing hope such reserving blush 
negative:
to the a of that on bad best restaurants hickenlooper war generating pays 
positive:
is this very never still hardly your girl material another hoot protagonist cheeks 
negative:
for in might pay 
positive:
you to with that clear not too but world mention winter 
positive:
the is you have and quality a far it way film fact summary exceptional natural soon filmmakers 
positive:
is you are and to a that say as its but go big much surprising deceitful 
positive:
is and a best does into more ; important characters uneasily nerve barry 
positive:
the and when has a light high actors fair once comics makers precision wang 
positive:
the is you a it like know being enjoys cinderella 


In [15]:
n_data = pd.read_table("train.tsv", header = 0)

In [16]:
n_data = np.array(n_data).tolist()

In [17]:
ref_data = ["" for i in range(8544)]
count = 0
for j in range(8544): 
    for i in range(count,len(n_data)):
        if(n_data[i][1] == j + 1):
            ref_data[j] = n_data[i][-2]
            count = i + 1
            break

In [18]:
ref_label = [0 for i in range(8544)]
count = 0
for j in range(8544): 
    for i in range(count,len(n_data)):
        if(n_data[i][1] == j + 1):
            ref_label[j] = n_data[i][-1]
            count = i + 1
            break

In [19]:
ref_label = ['1' if ref_label[i] >= 2 else '-1' for i in range(8544)]

In [20]:
ref_token_list = [[] for i in range(8544)]
for i in range(8544):
    ref_data[i] = re.sub('[.,]','',ref_data[i])
    ref_token_list[i] = word_tokenize(ref_data[i])